Creating Search using tfidf

In [1]:
import json
import re
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
tokenizer = TreebankWordTokenizer()

In [3]:
baby_shark = "Let's face it: Baby Shark is an undeniable force. James invites his guests, Sophie Turner, playing the role of Mommy Shark, and Josh Groban, taking on the role of Daddy Shark, for the definitive performance of this global phenomenon. More Late Late Show: \
Subscribe: http://bit.ly/CordenYouTube \
Watch Full Episodes: http://bit.ly/1ENyPw4 \
Facebook: http://on.fb.me/19PIHLC \
Twitter: http://bit.ly/1Iv0q6k \
Instagram: http://bit.ly/latelategram \
Watch The Late Late Show with James Corden weeknights at 12:35 AM ET/11:35 PM CT. Only on CBS. \
Get new episodes of shows you love across devices the next day, stream live TV, and watch full seasons of CBS fan favorites anytime, anywhere with CBS All Access. Try it free! http://bit.ly/1OQA29B \
---\
Each week night, THE LATE LATE SHOW with JAMES CORDEN throws the ultimate late night after party with a mix of celebrity guests, edgy musical acts, games and sketches. Corden differentiates his show by offering viewers a peek behind-the-scenes into the green room, bringing all of his guests out at once and lending his musical and acting talents to various sketches. Additionally, bandleader Reggie Watts and the house band provide original, improvised music throughout the show. Since Corden took the reigns as host in March 2015, he has quickly become known for generating buzzworthy viral videos, such as Carpool Karaoke."


In [4]:
#data= tokenizer.tokenize(baby_shark.lower())
#https?\:\/\/[A-Z, a-z, \., \/, 0-9]+
def tokenize2(text):
    text= text.lower()
    return re.findall(r'\w+', text)
# data=tokenize2(baby_shark)
# print(data)

In [5]:
title_to_text={}
title_to_index={}


with open('./data/medium/medium-data-small.json') as f:
    data = json.load(f)
i=0
for medium in data:
    title_to_index[medium["title"]]=i
    title_to_text[medium["title"]] = tokenize2(medium["text"])
    i+=1

    

def getLink(query):    
    if(query == "hi"):
        return [data[0]["link"],data[1]["link"],data[2]["link"],data[3]["link"],data[4]["link"]]
    else:
        return 0  

In [6]:
print(title_to_index[data[2]["title"]])

2


In [7]:
print(title_to_text[data[2]["title"]])

['machine', 'learning', 'code', 'can', 'be', 'notoriously', 'difficult', 'to', 'debug', 'with', 'bugs', 'that', 'are', 'expensive', 'to', 'chase', 'even', 'for', 'simple', 'feedforward', 'neural', 'networks', 'you', 'often', 'have', 'to', 'make', 'several', 'decisions', 'around', 'network', 'architecture', 'weight', 'initialization', 'and', 'network', 'optimization', 'all', 'of', 'which', 'can', 'lead', 'to', 'insidious', 'bugs', 'in', 'your', 'machine', 'learning', 'code', 'as', 'chase', 'roberts', 'wrote', 'in', 'an', 'excellent', 'piece', 'on', 'how', 'to', 'unit', 'test', 'machine', 'learning', 'code', 'his', 'frustrations', 'stemmed', 'from', 'common', 'traps', 'like', 'so', 'what', 'is', 'to', 'be', 'done', 'about', 'it', 'this', 'article', 'will', 'provide', 'a', 'framework', 'to', 'help', 'you', 'debug', 'your', 'neural', 'networks', 'feel', 'free', 'to', 'skip', 'to', 'a', 'particular', 'section', 'or', 'read', 'through', 'below', 'please', 'note', 'we', 'do', 'not', 'cover', 

In [8]:
n_feats = 5000
doc_by_vocab = np.empty([len(data), n_feats])

def build_vectorizer(max_features, stop_words, max_df=0.8, min_df=10, norm='l2'):
    """Returns a TfidfVectorizer object with the above preprocessing properties.
    
    Note: This function may log a deprecation warning. This is normal, and you
    can simply ignore it.
    
    Params: {max_features: Integer,
             max_df: Float,
             min_df: Float,
             norm: String,
             stop_words: String}
    Returns: TfidfVectorizer
    """
    # YOUR CODE HERE
    return TfidfVectorizer(stop_words=stop_words, max_df=max_df, min_df=min_df,max_features=max_features, norm=norm)
    
tfidf_vec = build_vectorizer(n_feats, "english")
doc_by_vocab = tfidf_vec.fit_transform([d['text'] for d in data]).toarray()
baby_shark_vec = tfidf_vec.transform(baby_shark).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

/Users/Mobryan/Desktop/CS4300/assignment5/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1039: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [9]:
def cos_sim(query,doc):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-9-1f13aa4a9e6f>, line 2)